# Brain-Computer Interface Data Compression Analysis

This notebook provides a comprehensive analysis of different compression algorithms for neural data streams. We'll compare various compression techniques including lossless, lossy, and deep learning-based approaches to understand their trade-offs in terms of compression ratio, processing speed, and signal quality preservation.

## Overview

The analysis covers:
- **Lossless Compression**: Adaptive LZ, Dictionary-based compression
- **Lossy Compression**: Quantization, Wavelet transform
- **Deep Learning**: Autoencoder-based compression
- **Performance Metrics**: Compression ratio, latency, SNR preservation
- **Neural Data Characteristics**: Multi-channel recordings, temporal correlations

In [ ]:
# Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.fft import fft, fftfreq
import time
import warnings
warnings.filterwarnings('ignore')

# BCI Compression toolkit imports
import sys
sys.path.append('../src')
from bci_compression import NeuralCompressor, load_neural_data
from bci_compression.data_processing import (
    generate_synthetic_neural_data, 
    apply_bandpass_filter,
    apply_notch_filter
)
from bci_compression.algorithms import (
    AdaptiveLZCompressor,
    QuantizationCompressor, 
    WaveletCompressor,
    AutoencoderCompressor
)

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

## Load and Explore Neural Dataset

We'll generate synthetic neural data that mimics real brain-computer interface recordings. The synthetic data includes:
- Multiple neural channels (64 channels typical for BCI)
- Realistic neural oscillations (alpha, beta, gamma bands)
- Spike activity with physiological characteristics
- Background noise and cross-channel correlations

In [ ]:
# Generate synthetic neural data
n_channels = 64
n_samples = 30000  # 30 seconds at 1 kHz
fs = 1000.0  # Sampling frequency in Hz

print("Generating synthetic neural data...")
neural_data, metadata = generate_synthetic_neural_data(
    n_channels=n_channels,
    n_samples=n_samples,
    fs=fs,
    noise_level=0.1,
    spike_rate=5.0,
    seed=42
)

print(f"Data shape: {neural_data.shape}")
print(f"Data type: {neural_data.dtype}")
print(f"Data size: {neural_data.nbytes / 1e6:.2f} MB")
print(f"Duration: {metadata['duration']:.1f} seconds")
print(f"Sampling rate: {metadata['fs']:.0f} Hz")

# Basic statistics
print(f"\nData Statistics:")
print(f"Mean: {neural_data.mean():.4f}")
print(f"Std: {neural_data.std():.4f}")
print(f"Min: {neural_data.min():.4f}")
print(f"Max: {neural_data.max():.4f}")

# Create a DataFrame for easier analysis
time_vector = np.arange(n_samples) / fs
df_metadata = pd.DataFrame([metadata])
print(f"\nMetadata:")
print(df_metadata.T)

## Data Preprocessing

Neural data often requires preprocessing to remove artifacts and enhance signal quality. We'll apply:
- Bandpass filtering to isolate neural frequencies of interest
- Notch filtering to remove power line interference
- Normalization to standardize signal amplitudes across channels

In [ ]:
# Apply preprocessing to neural data
print("Applying preprocessing...")

# 1. Bandpass filter (1-100 Hz) to remove DC drift and high-frequency noise
filtered_data = apply_bandpass_filter(
    neural_data, 
    lowcut=1.0, 
    highcut=100.0, 
    fs=fs, 
    order=4
)

# 2. Notch filter to remove 50 Hz power line interference
filtered_data = apply_notch_filter(
    filtered_data,
    notch_freq=50.0,
    fs=fs,
    quality_factor=30.0
)

# 3. Z-score normalization per channel
preprocessed_data = np.zeros_like(filtered_data)
for ch in range(n_channels):
    channel_data = filtered_data[ch, :]
    preprocessed_data[ch, :] = (channel_data - channel_data.mean()) / channel_data.std()

print(f"Original data range: [{neural_data.min():.3f}, {neural_data.max():.3f}]")
print(f"Preprocessed data range: [{preprocessed_data.min():.3f}, {preprocessed_data.max():.3f}]")

# Visualize the effect of preprocessing
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot sample channel before and after preprocessing
ch_idx = 0
time_slice = slice(0, 5000)  # First 5 seconds

axes[0, 0].plot(time_vector[time_slice], neural_data[ch_idx, time_slice])
axes[0, 0].set_title('Original Signal (Channel 1)')
axes[0, 0].set_xlabel('Time (s)')
axes[0, 0].set_ylabel('Amplitude')

axes[0, 1].plot(time_vector[time_slice], preprocessed_data[ch_idx, time_slice])
axes[0, 1].set_title('Preprocessed Signal (Channel 1)')
axes[0, 1].set_xlabel('Time (s)')
axes[0, 1].set_ylabel('Normalized Amplitude')

# Power spectral density comparison
freqs = fftfreq(n_samples, 1/fs)[:n_samples//2]
original_psd = np.abs(fft(neural_data[ch_idx, :])[:n_samples//2])**2
preprocessed_psd = np.abs(fft(preprocessed_data[ch_idx, :])[:n_samples//2])**2

axes[1, 0].semilogy(freqs, original_psd)
axes[1, 0].set_title('Original PSD')
axes[1, 0].set_xlabel('Frequency (Hz)')
axes[1, 0].set_ylabel('Power')
axes[1, 0].set_xlim(0, 150)

axes[1, 1].semilogy(freqs, preprocessed_psd)
axes[1, 1].set_title('Preprocessed PSD')
axes[1, 1].set_xlabel('Frequency (Hz)')
axes[1, 1].set_ylabel('Power')
axes[1, 1].set_xlim(0, 150)

plt.tight_layout()
plt.show()

## Compression Algorithm Setup

We'll test different compression algorithms optimized for neural data characteristics:

1. **Lossless Algorithms**: Preserve exact signal reconstruction
   - Adaptive LZ compression
   - Dictionary-based compression

2. **Lossy Algorithms**: Allow controlled quality loss for higher compression
   - Quantization-based compression
   - Wavelet transform compression

3. **Deep Learning**: Neural network-based compression
   - Autoencoder compression

Each algorithm will be evaluated on compression ratio, processing speed, and signal quality preservation.

In [ ]:
# Initialize compression algorithms
algorithms = {
    'adaptive_lz': NeuralCompressor(algorithm='adaptive_lz', real_time=True),
    'quantization_8bit': QuantizationCompressor(bits=8, adaptive=True),
    'quantization_12bit': QuantizationCompressor(bits=12, adaptive=True),
    'wavelet_db4': WaveletCompressor(wavelet='db4', levels=5, threshold=0.1),
    'wavelet_db8': WaveletCompressor(wavelet='db8', levels=6, threshold=0.05),
}

# For autoencoder, we need to train it first
autoencoder = AutoencoderCompressor(latent_dim=32, epochs=50)

print("Compression algorithms initialized:")
for name, algo in algorithms.items():
    print(f"- {name}: {type(algo).__name__}")

print(f"- autoencoder: {type(autoencoder).__name__} (requires training)")

# Prepare test data (use preprocessed data)
test_data = preprocessed_data.astype(np.float32)  # Ensure consistent data type
print(f"\nTest data shape: {test_data.shape}")
print(f"Test data size: {test_data.nbytes / 1e6:.2f} MB")

## Compression Algorithm Evaluation

We'll evaluate each compression algorithm on multiple metrics:
- **Compression Ratio**: Original size / Compressed size
- **Compression Time**: Time to compress the data
- **Decompression Time**: Time to decompress the data
- **Signal Quality**: SNR and MSE after reconstruction
- **Throughput**: Data processing rate (MB/s)

In [ ]:
# Function to evaluate compression algorithm
def evaluate_compression(algorithm, data, algorithm_name):
    """Evaluate compression algorithm performance."""
    results = {}
    
    # Measure compression
    start_time = time.time()
    try:
        compressed_data = algorithm.compress(data)
        compression_time = time.time() - start_time
        
        # Measure decompression
        start_time = time.time()
        decompressed_data = algorithm.decompress(compressed_data)
        decompression_time = time.time() - start_time
        
        # Calculate metrics
        original_size = data.nbytes
        compressed_size = len(compressed_data)
        compression_ratio = original_size / compressed_size
        
        # Signal quality metrics
        if decompressed_data.shape != data.shape:
            # Handle shape mismatch (simplified reconstruction)
            decompressed_data = decompressed_data.reshape(data.shape)
        
        mse = np.mean((data - decompressed_data) ** 2)
        signal_power = np.var(data)
        snr_db = 10 * np.log10(signal_power / mse) if mse > 0 else float('inf')
        
        # Throughput
        total_time = compression_time + decompression_time
        throughput_mbps = (original_size / 1e6) / total_time if total_time > 0 else 0
        
        results = {
            'Algorithm': algorithm_name,
            'Compression_Ratio': compression_ratio,
            'Compression_Time_ms': compression_time * 1000,
            'Decompression_Time_ms': decompression_time * 1000,
            'Total_Time_ms': total_time * 1000,
            'Throughput_MBps': throughput_mbps,
            'SNR_dB': snr_db,
            'MSE': mse,
            'Original_Size_MB': original_size / 1e6,
            'Compressed_Size_MB': compressed_size / 1e6,
            'Status': 'Success'
        }
        
    except Exception as e:
        results = {
            'Algorithm': algorithm_name,
            'Compression_Ratio': 0,
            'Compression_Time_ms': 0,
            'Decompression_Time_ms': 0,
            'Total_Time_ms': 0,
            'Throughput_MBps': 0,
            'SNR_dB': 0,
            'MSE': float('inf'),
            'Original_Size_MB': data.nbytes / 1e6,
            'Compressed_Size_MB': 0,
            'Status': f'Failed: {str(e)}'
        }
    
    return results

# Run benchmarks
print("Running compression benchmarks...")
benchmark_results = []

for name, algorithm in algorithms.items():
    print(f"\nEvaluating {name}...")
    result = evaluate_compression(algorithm, test_data, name)
    benchmark_results.append(result)
    
    if result['Status'] == 'Success':
        print(f"  Compression ratio: {result['Compression_Ratio']:.2f}:1")
        print(f"  Processing time: {result['Total_Time_ms']:.2f} ms")
        print(f"  Throughput: {result['Throughput_MBps']:.2f} MB/s")
        print(f"  SNR: {result['SNR_dB']:.2f} dB")
    else:
        print(f"  {result['Status']}")

# Train and evaluate autoencoder separately
print(f"\nTraining autoencoder on subset of data...")
training_data = test_data[:, :10000]  # Use first 10 seconds for training
autoencoder.fit(training_data)

print("Evaluating autoencoder...")
result = evaluate_compression(autoencoder, test_data, 'autoencoder')
benchmark_results.append(result)

if result['Status'] == 'Success':
    print(f"  Compression ratio: {result['Compression_Ratio']:.2f}:1")
    print(f"  Processing time: {result['Total_Time_ms']:.2f} ms")
    print(f"  Throughput: {result['Throughput_MBps']:.2f} MB/s")
    print(f"  SNR: {result['SNR_dB']:.2f} dB")
else:
    print(f"  {result['Status']}")

# Create results DataFrame
results_df = pd.DataFrame(benchmark_results)
print(f"\nBenchmark completed! {len(benchmark_results)} algorithms evaluated.")

## Results Analysis and Visualization

Now we'll analyze the benchmark results and create visualizations to compare the performance of different compression algorithms across various metrics.

In [ ]:
# Display results table
successful_results = results_df[results_df['Status'] == 'Success'].copy()

print("Compression Algorithm Performance Summary:")
print("=" * 80)
display_cols = ['Algorithm', 'Compression_Ratio', 'Total_Time_ms', 'Throughput_MBps', 'SNR_dB']
print(successful_results[display_cols].round(2))

# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

if len(successful_results) > 0:
    # 1. Compression Ratio Comparison
    ax1 = axes[0, 0]
    bars1 = ax1.bar(successful_results['Algorithm'], successful_results['Compression_Ratio'])
    ax1.set_title('Compression Ratio Comparison')
    ax1.set_ylabel('Compression Ratio')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}:1', ha='center', va='bottom')
    
    # 2. Processing Time Comparison
    ax2 = axes[0, 1]
    bars2 = ax2.bar(successful_results['Algorithm'], successful_results['Total_Time_ms'])
    ax2.set_title('Total Processing Time')
    ax2.set_ylabel('Time (ms)')
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. Throughput Comparison
    ax3 = axes[0, 2]
    bars3 = ax3.bar(successful_results['Algorithm'], successful_results['Throughput_MBps'])
    ax3.set_title('Throughput Comparison')
    ax3.set_ylabel('Throughput (MB/s)')
    ax3.tick_params(axis='x', rotation=45)
    
    # 4. Signal Quality (SNR)
    ax4 = axes[1, 0]
    bars4 = ax4.bar(successful_results['Algorithm'], successful_results['SNR_dB'])
    ax4.set_title('Signal Quality (SNR)')
    ax4.set_ylabel('SNR (dB)')
    ax4.tick_params(axis='x', rotation=45)
    
    # 5. Compression Ratio vs SNR Trade-off
    ax5 = axes[1, 1]
    scatter = ax5.scatter(successful_results['Compression_Ratio'], successful_results['SNR_dB'], 
                         s=100, alpha=0.7)
    ax5.set_xlabel('Compression Ratio')
    ax5.set_ylabel('SNR (dB)')
    ax5.set_title('Compression vs Quality Trade-off')
    
    # Add labels for each point
    for i, row in successful_results.iterrows():
        ax5.annotate(row['Algorithm'], 
                    (row['Compression_Ratio'], row['SNR_dB']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    # 6. Efficiency Plot (Compression Ratio vs Throughput)
    ax6 = axes[1, 2]
    scatter2 = ax6.scatter(successful_results['Compression_Ratio'], successful_results['Throughput_MBps'],
                          s=100, alpha=0.7, c=successful_results['SNR_dB'], cmap='viridis')
    ax6.set_xlabel('Compression Ratio')
    ax6.set_ylabel('Throughput (MB/s)')
    ax6.set_title('Efficiency: Compression vs Speed')
    
    # Add colorbar for SNR
    cbar = plt.colorbar(scatter2, ax=ax6)
    cbar.set_label('SNR (dB)')
    
    # Add labels
    for i, row in successful_results.iterrows():
        ax6.annotate(row['Algorithm'], 
                    (row['Compression_Ratio'], row['Throughput_MBps']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)

else:
    # If no successful results, show message
    for ax in axes.flat:
        ax.text(0.5, 0.5, 'No successful compression results to display', 
                ha='center', va='center', transform=ax.transAxes, fontsize=12)
        ax.set_title('No Data')

plt.tight_layout()
plt.show()

# Performance ranking
if len(successful_results) > 0:
    print("\nAlgorithm Rankings:")
    print("=" * 50)
    
    # Rank by compression ratio
    ranking_compression = successful_results.nlargest(3, 'Compression_Ratio')
    print("Top 3 by Compression Ratio:")
    for i, (_, row) in enumerate(ranking_compression.iterrows()):
        print(f"  {i+1}. {row['Algorithm']}: {row['Compression_Ratio']:.2f}:1")
    
    # Rank by speed
    ranking_speed = successful_results.nlargest(3, 'Throughput_MBps')
    print("\nTop 3 by Processing Speed:")
    for i, (_, row) in enumerate(ranking_speed.iterrows()):
        print(f"  {i+1}. {row['Algorithm']}: {row['Throughput_MBps']:.2f} MB/s")
    
    # Rank by signal quality
    ranking_quality = successful_results.nlargest(3, 'SNR_dB')
    print("\nTop 3 by Signal Quality:")
    for i, (_, row) in enumerate(ranking_quality.iterrows()):
        print(f"  {i+1}. {row['Algorithm']}: {row['SNR_dB']:.2f} dB")
    
    # Overall score (weighted combination)
    successful_results['Overall_Score'] = (
        0.4 * successful_results['Compression_Ratio'] / successful_results['Compression_Ratio'].max() +
        0.3 * successful_results['Throughput_MBps'] / successful_results['Throughput_MBps'].max() +
        0.3 * successful_results['SNR_dB'] / successful_results['SNR_dB'].max()
    )
    
    ranking_overall = successful_results.nlargest(3, 'Overall_Score')
    print("\nTop 3 Overall (Weighted Score):")
    for i, (_, row) in enumerate(ranking_overall.iterrows()):
        print(f"  {i+1}. {row['Algorithm']}: Score {row['Overall_Score']:.3f}")
        
else:
    print("\nNo successful results to rank.")